In [27]:
# Initialize notebook
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
pd.options.mode.chained_assignment = None  # default='warn'

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = True

In [2]:
# Setup template and html config for plotly figures.
import plotly.graph_objects as go

TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=True, # must be True to auto-scale when resizing
        margin=dict(l=50, r=25, b=40, t=20, pad=4),
        font=dict(
            family='sans-serif',
            size=14
        ),
        hovermode='closest',
        xaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        yaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            ticks='outside',
            minor=dict(
                ticks='outside',
            ),
            zeroline=False,
        ),
        hoverlabel=dict(align='left'),
    )
)

TFB_CONFIG = {
    'responsive': True, # must be True to auto-scale when resizing
    'autosizable': True, # doesn't impact auto rescaling
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}


In [56]:
# plots Pathfinder PWL XP along with theoretical value

x  = np.array([-4,-3,-2,-1, 0, 1, 2, 3, 4])
y  = np.array([18,21,26,32,40,48,60,72,90])
#yf = np.array([10,15,20,30,40,60,80,120,160])*np.power(13/12, -2*x)
yf = 40*np.power(2, 0.5*x)*np.power(13/12, -2*x)

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='relative level',
            range=[-4.5,4.5],
            tick0=0, dtick=1,
        ),
        yaxis=dict(
            title_text='XP',
            range=[0, 100],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

fig.add_trace(go.Scatter(
    x=x, 
    y=y,
    mode='markers+lines', 
    name='PWL',
    hovertemplate=
        'level %{x}<br>'+
        'PWL %{y:.0f} XP' + 
        '<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=x,
    y=yf,
    mode='markers+lines',
    name='theory',
    hovertemplate=
        'level %{x}<br>'+
        'theory %{y:.0f} XP' + 
        '<extra></extra>'
))

fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-pf-xp-theory-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    #fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-pf-xp-theory-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    #fig_soup.div['style'] = 'aspect-ratio: 600/550;'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [47]:
# plots D&D XP vs CR along with an exponential fit.
dfD0 = pd.read_csv('../../assets/data/dmg-monster-xp.csv') # 'CR','XP'
dfD0 = dfD0[dfD0['CR'].between(1,30)]

x = dfD0['CR']
y = dfD0['XP']
ylog = np.log(dfD0['XP'])/np.log(2)

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0.5,30.5],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='XP',
            type='log',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

fig.add_trace(go.Scatter(
    x=x, 
    y=y,
    mode='markers+lines', 
    name='DMG',
    hovertemplate=
        'CR %{x}<br>'+
        'DMG %{y:.0f} XP' + 
        '<extra></extra>'
))

# add fit line
coefs = np.polyfit(x[2:-1], ylog[2:-1], 1)
#name = '$\mathrm{fit} = 2^{' + '{:.3f} LV + {:.3f}'.format(coefs[0], coefs[1]) + '}$'
name = 'fit'
poly = np.poly1d(coefs)
fig.add_trace(go.Scatter(
    x=x,
    y=np.power(2, poly(x)),
    mode='lines',
    name=name,
    hovertemplate=
        'CR %{x}<br>'+
        'fit %{y:.0f} XP' + 
        '<extra></extra>'
))

fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-dnd-xp-fit-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    #fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10))
    file_name = f'./fig-dnd-xp-fit-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    #fig_soup.div['style'] = 'aspect-ratio: 600/550;'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))